In [84]:
# Import libraries

import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV

In [85]:
df = pd.read_csv("../cleaned_data/train_data.csv") # Preprocessed training data
df

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education_Bachelor's,...,HasDependents_No,HasDependents_Yes,LoanPurpose_Auto,LoanPurpose_Business,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other,HasCoSigner_No,HasCoSigner_Yes,Default
0,45,72928,23252,516,72,1,24.53,24,0.79,0,...,1,0,0,0,1,0,0,0,1,0
1,41,65437,15975,328,81,4,19.00,60,0.69,0,...,1,0,0,0,0,1,0,1,0,0
2,34,147083,73467,607,67,1,8.20,36,0.66,0,...,0,1,0,1,0,0,0,0,1,0
3,25,97594,165559,489,115,4,21.12,24,0.21,0,...,0,1,0,0,0,0,1,1,0,0
4,22,143415,197038,424,15,3,12.84,36,0.61,0,...,0,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163416,56,95731,78549,580,53,2,13.39,48,0.54,1,...,1,0,0,0,0,0,1,0,1,0
163417,32,46410,27511,799,57,4,14.20,48,0.41,0,...,0,1,1,0,0,0,0,0,1,0
163418,54,142603,177416,519,20,3,10.69,12,0.47,0,...,1,0,0,0,0,1,0,1,0,0
163419,50,99826,161987,613,61,3,12.31,48,0.56,0,...,0,1,0,0,0,0,1,0,1,0


In [86]:
default = np.array(df["Default"])
df.drop(columns=["Default"], inplace=True)

df["LoanTerm"] = df["LoanTerm"] / 12

We now train the adaboost classifier on the training data, using a random forst as the estimator. The hyperparameters are decided using BayesSearchCV and the best estimator is used to make the final predictions.

In [ ]:
rfc = RandomForestClassifier(
    n_estimators=11,
    max_depth=5,
    criterion="gini",
    random_state=42
)

abc = AdaBoostClassifier(estimator=rfc, random_state=42)

In [107]:
params = {
    "n_estimators": [21],
    "learning_rate": (0.7, 0.9, "log-uniform")
}

grid_search = BayesSearchCV(abc, params, cv=4, n_iter=3, n_jobs=-1, random_state=42, verbose=2)

In [108]:
grid_search.fit(df, default)
model = grid_search.best_estimator_ # Get the best model

Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits


In [109]:
model

AdaBoostClassifier(estimator=RandomForestClassifier(max_depth=5,
                                                    n_estimators=11,
                                                    random_state=42),
                   learning_rate=0.7827968124693183, n_estimators=21,
                   random_state=42)

In [110]:
test_df = pd.read_csv("../cleaned_data/test_data.csv") # Validation data for testing the trained model

y_true = np.array(test_df["Default"])
test_df.drop(columns=["Default"], inplace=True)

# Accuracy of model on validation data
y_pred = model.predict(test_df)
accuracy_score(y_true, y_pred)

0.8843743880947719

In [111]:
test_df = pd.read_csv("../data/test.csv") # Test data

ids = np.array(test_df["LoanID"])
test_df.drop(columns="LoanID", inplace=True)

# Do label encoding on test data
test_df = pd.get_dummies(test_df, columns = test_df.select_dtypes(include=['object','category']).columns,dtype=int)

Now, we generate the csv file for the predictions on the actual test data. It achieves a score of 0.88768 on Kaggle.

In [ ]:
submission = []

test_df["LoanTerm"] /= 12

y_pred = model.predict(test_df)

submission.append(ids)
submission.append(y_pred)

submission = np.array(submission).T
submission = pd.DataFrame(submission)

submission.to_csv("adaboost_submission.csv", header=["LoanID", "Default"], index=False)